In [17]:
# Libraries Loading
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import xgboost as xgb
from matplotlib.ticker import MaxNLocator
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (average_precision_score, confusion_matrix,
                             f1_score, log_loss, precision_score, recall_score,
                             roc_auc_score)
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import roc_auc_score
import pickle

# read in data from PCA
X_train_pca = pd.read_pickle('PCA_X_train.pkl')
y_train_pca = pd.read_pickle('PCA_y_train.pkl')
X_test_pca = pd.read_pickle('PCA_X_test.pkl')
y_test_pca = pd.read_pickle('PCA_y_test.pkl')


# read in data from Autoencoder
X_train_autoencoder = pd.read_pickle('PCA_X_train.pkl')
y_train_autoencoder = pd.read_pickle('PCA_y_train.pkl')
X_test_autoencoder = pd.read_pickle('PCA_X_test.pkl')
y_test_autoencoder = pd.read_pickle('PCA_y_test.pkl')

In [2]:
url = 'https://drive.google.com/file/d/1E9bFoMAwMVPMkZHmMQfWvSgaWoCLtKp4/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_test_autoencoder = pd.read_pickle(url)

url = 'https://drive.google.com/file/d/1dknbQLJhQsbSFDNIpwjxZQwPc3fF14yF/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_train_autoencoder = pd.read_pickle(url)


url = 'https://drive.google.com/file/d/105vmXogIQU8srmtdzfCHYQ9G-T7Iu5hl/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
X_test_autoencoder = pd.read_pickle(url)

url = 'https://drive.google.com/file/d/1QemU274E0c14R8uknuipP20rNgDSLzcg/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
X_train_autoencoder = pd.read_pickle(url)

In [3]:
y_test_autoencoder.shape

(103495,)

In [4]:
y_train_autoencoder.shape

(796896,)

In [5]:
X_test_autoencoder.shape

(103495, 10)

In [6]:
X_train_autoencoder.shape

(796896, 10)

## XGBoost Classifier (Autoencoder)

In [8]:
# Define the XGBoost model
xgb_model = xgb.XGBClassifier()

# Define the hyperparameters to be tuned
parameters = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 150]
}


# Create a GridSearchCV object
xgb_grid = GridSearchCV(xgb_model, parameters, cv=3, n_jobs=-1)

# Fit the GridSearchCV object to the training data
xgb_grid.fit(X_train_autoencoder, y_train_autoencoder)
             
#the best model in 
print('Best hyperparameters:', xgb_grid.best_params_)
print('Best cross-validation AUC score:', xgb_grid.best_score_)

Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 150}
Best cross-validation AUC score: 0.9981465586475524


In [12]:
best_params = xgb_grid.best_params_
best_params

{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 150}

In [13]:
#retrain the model by using all data
xgboost_retrain = xgb.XGBClassifier(learning_rate =  best_params['learning_rate'], max_depth =  best_params['max_depth'], n_estimators = best_params['n_estimators']) #change it after trading
xgboost_retrain.fit(X_train_autoencoder, y_train_autoencoder)
yhat_test = xgboost_retrain.predict(X_test_autoencoder)

In [15]:
#AUC performance in test set
ROC_AUC_XGBOOST = roc_auc_score(y_test_autoencoder, yhat_test)
print('ROC AUC for XGBoost in test dataset : {:.4f}'.format(ROC_AUC_XGBOOST))

ROC AUC for XGBoost in test dataset : 0.9835


In [18]:
#convert to pickle file for latter running
pickle.dump(xgboost_retrain, open('XGBoostModel.pkl', 'wb'))